In [ ]:
!pip install transformers torch pandas

In [18]:
pip install accelerate -U

In [6]:
!pip install transformers[torch]
!pip install evaluate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.1 MB/s eta 0:00:00


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import torch

In [2]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import torch
import pandas as pd

# Load your data
df1=pd.read_excel('Winter23_logs.xlsx').drop(0)

df1['Outcome']=df1['Outcome'].str.lower()
df1.head()
df=pd.DataFrame({'Outcome': df1['Outcome']})
df=df.dropna()


In [5]:
# Create a language modeling dataset
text_data = list(df['Outcome'])
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
tokenized_data = tokenizer(text_data, return_tensors="pt", padding=True, truncation=True)

# Fine-tune the language model (GPT-2)
config = GPT2Config.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2", config=config)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

# Extract input_ids
input_ids = tokenized_data['input_ids']

# Assuming you have a train-validation split
input_ids_train, input_ids_val = train_test_split(
    input_ids, test_size=0.1, random_state=42
)

# Create DataLoader
train_data = torch.utils.data.TensorDataset(input_ids_train)
val_data = torch.utils.data.TensorDataset(input_ids_val)

train_loader = DataLoader(train_data, batch_size=4, shuffle=True)
val_loader = DataLoader(val_data, batch_size=4, shuffle=False)


# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(3):  # You may need more epochs based on your data size and complexity
    model.train()
    for batch in tqdm(train_loader, desc="Training"):
        optimizer.zero_grad()
        inputs = {"input_ids": batch[0].to(device)}
       # inputs = {key: value.to(device) for key, value in batch.items()}
        outputs = model(**inputs, labels=inputs["input_ids"])
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# Save the fine-tuned language model
model.save_pretrained("fine_tuned_language_model")



vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Training: 100%|██████████| 767/767 [03:15<00:00,  3.93it/s]


In [3]:
# Load the fine-tuned language model for classification
model = GPT2LMHeadModel.from_pretrained("fine_tuned_language_model")

In [4]:
import numpy as np
import pandas as pd

# from nltk.tokenize import sent_tokenize

from pathlib import Path
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader
from transformers import GPT2ForSequenceClassification, AutoModelForSequenceClassification,GPT2Config, GPT2LMHeadModel, AutoTokenizer,TrainingArguments, Trainer, DataCollatorWithPadding
import re

device = 'cuda' if torch.cuda.is_available() else 'cpu'


In [5]:
df2=pd.read_csv('cleaned_winter_data.csv')
df2 = df2.drop('Requires further investigation to resolve?', axis=1)
df2['Description of problem'] = (df2['Received from:'] + ' ' + df2['Description of problem'].fillna('')).astype(str)
dataset=df2.reindex(['Description of problem','Sent staff to room?'], axis=1)
dataset.head()

,Description of problem,Sent staff to room?
0,OI 8170 jr. station enable request from kyle,0
1,BA 1180 wanted to know if his log in would wor...,0
2,UC 85 log in request,0
3,ES 4001 mike showing prof how intercom works,0
4,BL 313 prof testing powerpoint for next week,0


In [6]:
import string
def remove_punctuation_and_quotes(text):
    return ''.join(char.lower() for char in text if char not in string.punctuation + "'\"")
dataset['Description of problem'] = dataset['Description of problem'].apply(remove_punctuation_and_quotes)
dataset['Sent staff to room?'] = dataset['Sent staff to room?'].map({'1': 1, '0': 0})

In [7]:
#This is the custom dataset
class ClassificationDataset(Dataset):
    def __init__(self,  tokenizer) -> None:
        super().__init__()

        self.dataset = dataset
        self.tokenizer = tokenizer
        self.max_sentence_length = np.max([len(d) for d in self.dataset['Description of problem']])
        self.tokenizer.pad_token = self.tokenizer.eos_token

    def __len__(self):
        return len(self.dataset['Description of problem'])

    def __getitem__(self, index):
        text = self.dataset['Description of problem'][index]
        label = self.dataset['Sent staff to room?'][index]
        # Tokenize the text using your custom tokenizer
        tokenized = self.tokenizer(text, padding = 'max_length', truncation = True, max_length = self.max_sentence_length)

        return {'input_ids': tokenized['input_ids'], 'labels': torch.tensor(label),'attention_mask': tokenized['attention_mask']}

In [8]:
data_full = ClassificationDataset(tokenizer = AutoTokenizer.from_pretrained("gpt2"))

# Create the data collator
data_collator = DataCollatorWithPadding(tokenizer=data_full.tokenizer)

In [9]:
import evaluate
metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [10]:
train_data, val_data = train_test_split(data_full, test_size=0.2)

In [11]:
# Create the training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    logging_steps=100,
    evaluation_strategy="epoch"
)

In [12]:
# Fine-tune the model
trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=data_full.tokenizer,
    train_dataset=train_data,
    eval_dataset=val_data,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [13]:
torch.cuda.empty_cache()

In [14]:
trainer.train()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


ValueError: ignored